# Data Science Notebook

Build, train and serialize the model

# Import packages

In [1]:
# load data
from submodules.load_data import load_data

# data manipulation
import pandas as pd

# data splitting
from sklearn.model_selection import train_test_split

# data preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

# model
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# k-fold cross validation
from sklearn.model_selection import cross_validate

# saving models
import joblib

# performance
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

# Load the data

Load semi-colon separated data from disk

In [2]:
data = load_data()

# Create a Test Dataset
> uses scikit-learn

Performing this early minimizes generalization and bias you may inadvertently apply to your system.
Simply put, a test set of data involves: picking ~20% of the instances randomly and setting them aside.

Some considerations for sampling methods that generate the test set:
1. you don't want your model to see the entire dataset
1. you want to be able to fetch new data for training
1. you want to maintain the same percentage of training data against the entire dataset
1. you want a representative training dataset (~7% septic positive)

https://realpython.com/train-test-split-python-data/

In [3]:
# sets 20% of the data aside for testing, sets the random number generate to it always generates the same shuffled indicies
# x = 2 dimensional array with inputs
# X_train is the training part of the first sequence (x)
# X_test is the test part of the first sequence (x)
# y = 1 dimensional array with outputs
# y_train is the labeled training part of the second sequence
# y_test is the labeled test part of the second sequence
# axis Whether to drop labels from the index (0 or ‘index’) or columns (1 or ‘columns’)
# test_size is the amount of the total dataset to set aside for testing = 10%
# random state fixes the randomization so you get the same results each time
# Shuffle before the data is split, it is shuffled
# stratified splitting keeps the proportion of y values trhough the train and test sets
X_train, X_test, y_train, y_test = \
    train_test_split(data.drop("isSepsis", axis=1),
    data["isSepsis"], test_size=0.1,
    random_state=42, stratify=data["isSepsis"])

# Prepare data for Machine Learning
Instead of preparing data manually, write functions to:
1. reproduce transformations easily on any dataset (e.g., data refresh)
1. builds a library of functions to reuse in future projects
1. use functions in live stream to transform new data before inferencing

## Data cleaning
1. split numerical attributes
    1. transform current and future null values
    1. drop under-represented attributes (<7k)
    1. impute median for missing attributes (>7k)

### Copy Numeric Training Data, drop categorical and under-represented data
1. under-rep. Unit1
1. under-rep. Unit2
1. under-rep. Gender
1. drop Bilirubin_direct
1. drop EtCO2
1. drop Fibrinogen
1. drop Troponin I
1. drop Lactate
1. drop SaO2
1. drop FiO2

In [9]:
# data_num = X_train.copy()
# .drop also creates a copy without the categorical attributes
# drop non-biological indicators
data_num = X_train.drop(["SBP",
                         "DBP",
                         "EtCO2",
                         "BaseExcess",
                         "Magnesium",
                         "Bilirubin_direct",
                         "EtCO2",
                         "Fibrinogen",
                         "TroponinI",
                         "Lactate",
                         "SaO2",
                         "FiO2",
                         "Age",
                         "Gender",
                         "Unit1",
                         "Unit2",
                         "HospAdmTime",
                         "ICULOS"], axis=1)
data_num.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,pH,...,Glucose,Magnesium,Phosphate,Potassium,Bilirubin_total,Hct,Hgb,PTT,WBC,Platelets
17166,90.0,100.0,37.3,108.5,79.5,67.5,10.0,0.0,23.0,7.33,...,90.0,NaN,NaN,3.7,NaN,33.7,11.6,34.2,24.0,157.0
8962,91.0,NaN,NaN,137.0,82.0,NaN,21.0,NaN,27.0,NaN,...,102.0,2.1,3.8,4.3,0.7,40.4,13.9,38.6,8.3,209.0
10453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,...,99.0,2.1,4.2,5.1,0.5,31.4,9.6,NaN,7.2,585.0
19087,66.0,100.0,37.6,124.0,78.0,58.0,16.0,NaN,NaN,NaN,...,130.0,1.7,NaN,4.1,NaN,26.9,9.0,NaN,14.2,179.0
5328,103.0,100.0,NaN,108.0,82.0,68.0,NaN,NaN,25.0,NaN,...,94.0,1.7,2.4,3.5,NaN,23.7,8.4,30.5,11.2,167.0


### Transform missing values from numeric data

In [70]:
# create simpleimputer instance
# replace attributes missing values with median of the attribute
num_imputer = SimpleImputer(strategy="median")

# fit applies the imputer to ALL numeric data in case new data includes null values
# when system goes live
# results are stored in a imputer.statistics_ value
num_imputer.fit(data_num)
num_imputer.statistics_

array([ 82.  ,  98.  ,  36.8 , 120.  ,  80.  ,  62.  ,  18.  ,   0.  ,
        24.  ,   7.39,  41.  ,  32.  ,  16.  ,  72.  ,   8.4 , 106.  ,
         0.9 , 122.  ,   2.  ,   3.4 ,   4.  ,   0.8 ,  31.8 ,  10.7 ,
        30.7 ,  10.1 , 193.  ,  63.25,  -6.08,  11.  ])

In [71]:
# apply the trained imputer to transform the training set replacing the
# missing values with learn medians
N = num_imputer.transform(data_num)
# result above is plain NumPy array with transformed features
# put back to a pandas DataFrame
num_tr = pd.DataFrame(N, columns=data_num.columns, index=data_num.index)
num_tr.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,pH,...,Potassium,Bilirubin_total,Hct,Hgb,PTT,WBC,Platelets,Age,HospAdmTime,ICULOS
17166,90.0,100.0,37.3,108.5,79.5,67.5,10.0,0.0,23.0,7.33,...,3.7,0.8,33.7,11.6,34.2,24.0,157.0,61.19,-3.62,4.0
8962,91.0,98.0,36.8,137.0,82.0,62.0,21.0,0.0,27.0,7.39,...,4.3,0.7,40.4,13.9,38.6,8.3,209.0,88.95,-62.83,16.0
10453,82.0,98.0,36.8,120.0,80.0,62.0,18.0,0.0,25.0,7.39,...,5.1,0.5,31.4,9.6,30.7,7.2,585.0,79.69,-0.02,14.0
19087,66.0,100.0,37.6,124.0,78.0,58.0,16.0,0.0,24.0,7.39,...,4.1,0.8,26.9,9.0,30.7,14.2,179.0,72.00,-31.88,9.0
5328,103.0,100.0,36.8,108.0,82.0,68.0,18.0,0.0,25.0,7.39,...,3.5,0.8,23.7,8.4,30.5,11.2,167.0,21.66,-0.03,21.0


### Copy Categorical Training Data

In [72]:
# data_cat = X_train.copy()
# .drop also creates a copy without the categorical attributes
data_cat = X_train[["Gender"]]
data_cat.head()

,Gender
17166,1
8962,1
10453,0
19087,0
5328,0


### Transform missing values from numeric data

In [73]:
# create simpleimputer instance
# replace missing using the most frequent value along each column
cat_imputer = SimpleImputer(strategy="most_frequent")

# fit applies the imputer to ALL numeric data in case new data includes null values
# when system goes live
# results are stored in a imputer.statistics_ value
cat_imputer.fit(data_cat)
cat_imputer.statistics_

array([1.])

In [74]:
# apply the trained imputer to transform the training set replacing the
# missing values with learn medians
C = cat_imputer.transform(data_cat)
# result above is plain NumPy array with transformed features
# put back to a pandas DataFrame
cat_tr = pd.DataFrame(C, columns=data_cat.columns, index=data_cat.index)
cat_tr.head()

,Gender
17166,1
8962,1
10453,0
19087,0
5328,0


## Feature Scaling
1. ML algorithms don't work well when numeric attributes have very different scales
    (e.g. HR max 184,  pH max 7.67)
1. Scaling target values is not necessary
1. Apply
    1. normalization (MinMaxScaler) bounds the values to a specific range (e.g. 0-1)
    1. standardization (StandardScaler) less affected by outliers does not bound to range

### Transformation Pipeline

1. Common to apply many transformation steps in a specific order

In [75]:
num_pipeline = Pipeline([
                        ('imputer', SimpleImputer(strategy='median')),
                        ('std_scaler', StandardScaler()),
                        ])

num_prepared = num_pipeline.fit_transform(data_num)

In [76]:
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OrdinalEncoder())
])

cat_prepared = cat_pipeline.fit_transform(data_cat)

## Full Data Pipeline

Single transformer to handle numeric and categorical columns using ColumnTransformer.

In [77]:
# get a list of numeric column names
num_attribs = list(data_num)
# get a list of categorical column names
cat_attribs = ["Gender"]

In [78]:
# construct the transformer
full_pipeline = ColumnTransformer([
    # transform number columns with num_pipeline defined earlier
    ("num", num_pipeline, num_attribs),
    # transform categorical columns with cat_pipeline defined earler
    ("cat", cat_pipeline, cat_attribs)
])

# only run the pipeline on the training as the test data will be applied during the evaluation stage with the final model
X_train_prepared = full_pipeline.fit_transform(X_train)

# Model Selection

![image](images/scikitlearn-choose-right-estimator.png)

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

## Classifier Comparison

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html?highlight=svm%20svc

1. [Linear Support Vector Machine "SVM" Support Vector Classifier "SVC"](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)
1. [Naive Bayes]()
1. [K-Neighbors Classifier](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification) implements learning based on the  nearest neighbors of each query point, where  is an integer value specified by the user.
1. [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.
1. [Logistic Regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) a linear model for classification
1. [Stochastic Gradient Descent "SGD" Classifier](https://scikit-learn.org/stable/modules/sgd.html#classification)
1. [Neural Network Multi-Layer Perceptron Classifier](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#multi-layer-perceptron) a supervised learning algorithm that learns a function  by training on a dataset
1. [XGBoost Classifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=xgbclassifier#xgboost.XGBClassifier)

## Scoring
https://scikit-learn.org/stable/modules/cross_validation.html#
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

Validation output readings"
    - fit_time - the time for fitting the estimator on the train set for each cv split
    - score_time - the time for scoring the estimator on the test set for each cv split
    - test_score - The score array for test scores on each cv split
    - train_score - The score array for train scores on each cv split.

Best practice to save every model you experiment with so you can come back easily to any model.
Save both the hyperparameters and trained parameter, as well as the cross-validation scores and predictions.
This will allow you to easily compare scores across model types. Use Pickle or joblib libraries.

## Linear SVM SVC

In [79]:
# select the SVM
lin_svm = svm.SVC(kernel='linear', # linear, poly, rbf, sigmoid, precomputed
                  C=1)  # Regularization parameter
# fit the model to the data
lin_svm.fit(X_train_prepared, y_train)
# configure the cross validation
cv_lin_svm = cross_validate(lin_svm, # estimator to fit
                            X_train_prepared, # data to fit
                            y_train, # target variable isSepsis
                            n_jobs=-1, # use all the processors in parallel
                            verbose=1, # verbosity level
                            cv=3, # splitting strategy to compute the score N consecutive times with different splits
                            scoring="f1", # for binary targets
                            return_train_score=True)

# display the scoring
cv_lin_svm

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.0s finished


{'fit_time': array([18.17841792, 17.97551918, 18.55389619]),
 'score_time': array([1.167032  , 1.17491698, 1.19085693]),
 'test_score': array([0.3935743 , 0.40119166, 0.3891675 ]),
 'train_score': array([0.39360639, 0.4       , 0.39640539])}

In [80]:
# save the model
joblib.dump(lin_svm, "models/lin_svm.pkl")
# reference to load the model
#lin_svm_loaded = joblib.load("model/lin_svm.pkl")

['models/lin_svm.pkl']

## Naive Bayes

In [81]:
gnb = GaussianNB()
gnb.fit(X_train_prepared, y_train)
cv_n_bayes = cross_validate(gnb, # estimator to fit
                            X_train_prepared, # data to fit
                            y_train, # target variable isSepsis
                            n_jobs=-1, # use all the processors in parallel
                            verbose=1, # verbosity level
                            cv=3, # splitting strategy to compute the score N consecutive times with different splits
                            scoring="f1", # for binary targets
                            return_train_score=True)

# display the scoring
cv_n_bayes

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.8s finished


{'fit_time': array([0.02321601, 0.02409697, 0.0232892 ]),
 'score_time': array([0.00846076, 0.00828505, 0.00849104]),
 'test_score': array([0.47810026, 0.44353183, 0.49229075]),
 'train_score': array([0.47433535, 0.47161804, 0.46266234])}

In [82]:
# save the model
joblib.dump(gnb, "models/gnb.pkl")
# reference to load the model
#gnb_loaded = joblib.load("model/gnb.pkl")

['models/gnb.pkl']

## K Nearest Neighbor Classification

In [83]:
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train_prepared, y_train)
cv_knn = cross_validate(knn,
                        X_train_prepared,
                        y_train,
                        n_jobs=-1,
                        verbose=1,
                        cv=3,
                        scoring="f1",
                        return_train_score=True)
cv_knn

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   24.2s finished


{'fit_time': array([0.01427698, 0.01236105, 0.01475787]),
 'score_time': array([9.0135572 , 9.59114289, 9.94566917]),
 'test_score': array([0.50137237, 0.48843663, 0.50693802]),
 'train_score': array([0.50978607, 0.5301314 , 0.49700874])}

In [84]:
# save the model
joblib.dump(knn, "models/knn.pkl")
# reference to load the model
#knn_loaded = joblib.load("model/knn.pkl")


['models/knn.pkl']

## Random Forest Classifier

In [85]:
rf = RandomForestClassifier(n_estimators=5, # The number of trees in the forest
                            verbose=1)
cv_rf = cross_validate(rf,
                       X_train_prepared,
                       y_train,
                       n_jobs=-1,
                       cv=3,
                       scoring="f1",
                       return_train_score=True)
cv_rf

{'fit_time': array([0.25918412, 0.23456287, 0.22348976]),
 'score_time': array([0.01054025, 0.01056385, 0.0103364 ]),
 'test_score': array([0.73068592, 0.74277457, 0.73342939]),
 'train_score': array([0.96341858, 0.96046662, 0.96475913])}

In [86]:
# save the model
joblib.dump(rf, "models/rf.pkl")
# reference to load the model
#rf_loaded = joblib.load("model/rf.pkl")

['models/rf.pkl']

## Logistic Regression
- [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression)

In [87]:
# set a variable to Logistic regression with verbosity
log_reg = LogisticRegression(solver="liblinear", # liblinear is ideal for small datasets
                             C=1, # regularization; smaller values specify stronger regularization
                             verbose=2)
log_reg.fit(X_train_prepared, y_train)
cv_log_reg = cross_validate(log_reg,
                            X_train_prepared, # attributes
                            y_train, # labels isSepsis
                            n_jobs=-1, # use all the processors in parallel
                            verbose=1, # verbosity level
                            cv=5, # splitting strategy to compute the score N consecutive times with different splits
                            scoring="f1", # for binary targets
                            return_train_score=True) # computationally expensive, whether to include training scores on parameters impact
cv_log_reg

[LibLinear]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.0s finished


{'fit_time': array([0.31530023, 0.31319308, 0.2987752 , 0.28334689, 0.16984296]),
 'score_time': array([0.00441599, 0.00363684, 0.00378966, 0.003901  , 0.00327706]),
 'test_score': array([0.49462366, 0.50228311, 0.49165402, 0.4893617 , 0.50306748]),
 'train_score': array([0.50057056, 0.49060943, 0.5026616 , 0.50171821, 0.49406358])}

In [ ]:
# save the model
joblib.dump(log_reg, "models/log_reg.pkl")
# reference to load the model
#log_reg_loaded = joblib.load("model/log_reg.pkl")

## SGDClassifier

advantages
- efficient

disadvantages
- sensitive to feature scaling

In [139]:
sgd_clf = SGDClassifier(loss="log", # logistic regression
                        penalty="elasticnet",
                        alpha=0.0001,
                        max_iter=1000,
                        shuffle=True,
                        learning_rate='optimal')
sgd_clf.fit(X_train_prepared, y_train)
cv_sgd_clf = cross_validate(sgd_clf,
                            X_train_prepared, # attributes
                            y_train, # labels isSepsis
                            n_jobs=-1, # use all the processors in parallel
                            verbose=1, # verbosity level
                            cv=5, # splitting strategy to compute the score N consecutive times with different splits
                            scoring="f1", # for binary targets
                            return_train_score=True) # computationally expensive, whether to include training scores on parameters impact

cv_sgd_clf

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


{'fit_time': array([0.36535096, 0.33189511, 0.70298386, 0.41409278, 0.35349202]),
 'score_time': array([0.00510001, 0.00801516, 0.00588202, 0.01588702, 0.00373793]),
 'test_score': array([0.47457627, 0.5129771 , 0.42492013, 0.5       , 0.50366032]),
 'train_score': array([0.47655356, 0.49246231, 0.4494829 , 0.51081594, 0.50764749])}

In [140]:
# serialize the model
joblib.dump(sgd_clf, "models/sgd_clf.pkl")
# reference load the model
#sgd_clf_loaded = joblib.load("model/sgd_clf.pkl")

['models/sgd_clf.pkl']

## MLP Classifier

In [89]:
nn = MLPClassifier(solver='sgd', # For small datasets, ‘lbfgs’ can converge faster and perform better.
                   activation='relu',
                   max_iter=5000, # The solver iterates until convergence
                   hidden_layer_sizes=(50,50,50,50), # The ith element represents the number of neurons in the ith hidden layer
                   verbose=0,
                   learning_rate="adaptive") #  keeps the learning rate constant to ‘learning_rate_init’ as long as training loss keeps decreasing
cv_nn = cross_validate(nn,
                       X_train_prepared,
                       y_train,
                       cv=3,
                       scoring="f1",
                       return_train_score=True,
                       verbose=1)
cv_nn

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  8.9min finished


{'fit_time': array([170.67743087, 175.3386929 , 186.25447607]),
 'score_time': array([0.02585816, 0.02407122, 0.0230298 ]),
 'test_score': array([0.58815959, 0.55924765, 0.55717762]),
 'train_score': array([0.99335233, 0.99905452, 0.99810845])}

In [90]:
# save the model
joblib.dump(nn, "models/nn.pkl")
# reference to load the model
#nn_loaded = joblib.load("model/nn.pkl")

['models/nn.pkl']

## XGBoost Classifier

Learning task parameter = https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters

In [154]:
xgboost = XGBClassifier(use_label_encoder=False, # removes user warning error
                        booster="gbtree")
cv_xgboost = cross_validate(xgboost,
                            X_train_prepared,
                            y_train,
                            cv=3,
                            scoring="f1",
                            return_train_score=True,
                            verbose=0)
cv_xgboost

{'fit_time': array([2.07458496, 1.16849709, 1.20430899]),
 'score_time': array([0.01188397, 0.00971794, 0.01256514]),
 'test_score': array([0.76801153, 0.78481013, 0.78847505]),
 'train_score': array([0.98352345, 0.98350254, 0.98224477])}

In [155]:
# save the model
joblib.dump(xgboost, "models/xgb.pkl")
# reference to load the model
#xgboost_loaded = joblib.load("models/xgboost.pkl")

['models/xgb.pkl']

## Fine-tune the model
Instead of messing with hyperparameters manually, GridSearchCV can be instructed to search hyperparameters
and uses cross validation to evaluate the possible combinations.

During this sampling cycle, you may go back to your pipeline and:
1. drop uninformative features
1. add extra features
1. clean up outliers

### Grid Search on Random Forest Classifier

[sklearn.ensemble.RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=random%20forest%20classifier#sklearn.ensemble.RandomForestClassifier)

In [93]:
rfc_param_grid = [
    {'n_estimators': [3, 10, 30, 100], # The number of trees in the forest.
     'max_features': [10, 15, 20, 30]}, # The number of features to consider when looking for the best split
    {'bootstrap': [False], # If False, the whole dataset is used to build each tree
     'n_estimators': [3,10],
     'max_features': [2, 3, 4]},
]

rfc = RandomForestClassifier()

# the grid search will explore 4 x 4 combinations of Random Forest Classifier combinations of
# n_estimators and max_features hyperparameters values
# Then will try 2 x 3 combinations of hyperparameter values in the second dictionary
# with the bootstrap set to false
# overall, gridsearch will explore 16 + 6 = 22 combinations of RFC hyperparameters and train each modele 3 times
rfc_grid_search = GridSearchCV(rfc,
                           rfc_param_grid,
                           cv=3,
                           scoring='f1',
                           return_train_score=True)

rfc_grid_search.fit(X_train_prepared, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid=[{'max_features': [10, 15, 20, 30],
                          'n_estimators': [3, 10, 30, 100]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='f1')

In [161]:
# print the best score
rfc_grid_search.best_score_

0.7699968013598836

In [94]:
# print the best combination of parameters
rfc_grid_search.best_params_

{'max_features': 10, 'n_estimators': 100}

In [95]:
# print the best estimator directly
rfc_grid_search.best_estimator_

RandomForestClassifier(max_features=10)

In [96]:
# display the importance scores next to the corresponding attributes
# from this you can drop less useful features
feature_importances = rfc_grid_search.best_estimator_.feature_importances_
attributes = num_attribs + cat_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.37735773040100734, 'ICULOS'),
 (0.04730115690408714, 'WBC'),
 (0.04592894807237559, 'HospAdmTime'),
 (0.0430434397655355, 'BUN'),
 (0.04199322766534698, 'Creatinine'),
 (0.04199091114740825, 'Hct'),
 (0.036617303219033906, 'Temp'),
 (0.03510400759777423, 'Hgb'),
 (0.0348865190227094, 'Platelets'),
 (0.027757209337781624, 'HR'),
 (0.02109932050910739, 'Resp'),
 (0.02097147290351952, 'Age'),
 (0.01987236598516874, 'SBP'),
 (0.019530999972846838, 'MAP'),
 (0.019358680289891166, 'DBP'),
 (0.01827024793709712, 'Glucose'),
 (0.017085521159405256, 'Calcium'),
 (0.013463495831909837, 'Potassium'),
 (0.013366043383783787, 'O2Sat'),
 (0.011796822775000576, 'Magnesium'),
 (0.011171343157348778, 'HCO3'),
 (0.010928940185258838, 'Phosphate'),
 (0.01014210901381482, 'Chloride'),
 (0.009339537480688625, 'PTT'),
 (0.00916774467836452, 'Bilirubin_total'),
 (0.008296114192904555, 'pH'),
 (0.008258409034425615, 'Alkalinephos'),
 (0.008072103732817054, 'AST'),
 (0.007909745356941606, 'PaCO2'),
 (0.007

### Grid Search on XGBoost
[xgboost.XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html?highlight=xgbclassifier#xgboost.XGBClassifier)

In [172]:
xgb_param_grid = {"n_estimators": [150, 200],
                  "max_delta_step": [0.1],
                  "subsample": [None, 0.5, 1],
                  "reg_lambda": [1, 1.1],
                  "alpha": [0, 0.1]}


xgb = XGBClassifier(use_label_encoder=False)

xgb_grid_search = GridSearchCV(xgb,
                           xgb_param_grid,
                           cv=3,
                           scoring='f1',
                           return_train_score=True)

xgb_grid_search.fit(X_train_prepared, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, us

In [173]:
# print the best score
xgb_grid_search.best_score_

0.7582144265457043

In [174]:
# print the best combination of parameters
xgb_grid_search.best_params_

{'alpha': 0,
 'max_delta_step': 0.1,
 'n_estimators': 200,
 'reg_lambda': 1.1,
 'subsample': None}

In [159]:
# print the best estimator directly
xgb_grid_search.best_estimator_

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0.1, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1.1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [160]:
feature_importances = xgb_grid_search.best_estimator_.feature_importances_
attributes = num_attribs + cat_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.17644046, 'ICULOS'),
 (0.15313622, 'BUN'),
 (0.12117299, 'WBC'),
 (0.06759909, 'Hgb'),
 (0.04747762, 'Creatinine'),
 (0.046846576, 'HospAdmTime'),
 (0.041359287, 'Platelets'),
 (0.026133139, 'Temp'),
 (0.023732722, 'HR'),
 (0.022136042, 'HCO3'),
 (0.017808491, 'Calcium'),
 (0.017374972, 'Hct'),
 (0.017354779, 'DBP'),
 (0.017209586, 'Chloride'),
 (0.016534038, 'PTT'),
 (0.015087422, 'Magnesium'),
 (0.015005713, 'Resp'),
 (0.014618657, 'pH'),
 (0.0141673405, 'O2Sat'),
 (0.013776249, 'Phosphate'),
 (0.0133996075, 'Potassium'),
 (0.013325603, 'SBP'),
 (0.013107742, 'Glucose'),
 (0.01257483, 'BaseExcess'),
 (0.011553415, 'MAP'),
 (0.01104402, 'Bilirubin_total'),
 (0.010610401, 'Alkalinephos'),
 (0.010555101, 'Age'),
 (0.009997199, 'PaCO2'),
 (0.0074839913, 'AST'),
 (0.0013767334, 'Gender')]

# Evaluate RFC model on the Test set

After tweaking the model, you can evaluate the final model on the test set.
1. get the predictors and labels from your test set
1. run the transformation pipeline with transform() < don't fit the test set
1. run the prediction

In [ ]:
# transform, DON'T fit the final data
X_test_ready = full_pipeline.transform(X_test)

In [163]:
final_model = rfc_grid_search.best_estimator_

final_predictions = final_model.predict(X_test_ready)

In [170]:
# view the percent of the predictions that were correct
accuracy_score(y_test, final_predictions)

0.952905535665106

# Evaluate XGB model on the Test set

In [171]:
final_model_x = xgb_grid_search.best_estimator_

x_final_predictions = final_model_x(X_test_ready)

TypeError: 'XGBClassifier' object is not callable

In [ ]:
# view the accuracy
accuracy_score(y_test, x_final_predictions)